In [46]:
!pip install geopy --quiet

In [47]:
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
import requests

# to stop SettingWithCopyWarning
pd.options.mode.chained_assignment = None

### Read in list of LGA names and codes
There are thousands of rows because each LGA is repeated many times. Need to extract just the unique LGA codes and names

In [48]:
df_lga = pd.read_csv("LGA_2019_VIC.csv")

In [49]:
print("Shape: ", df_lga.shape)
df_lga.head()

Shape:  (85014, 6)


,MB_CODE_2016,LGA_CODE_2019,LGA_NAME_2019,STATE_CODE_2016,STATE_NAME_2016,AREA_ALBERS_SQKM
0,20000009499,29499,No usual address (Vic.),2,Victoria,0.0000
1,20000010000,20110,Alpine (S),2,Victoria,0.0127
2,20000021000,20110,Alpine (S),2,Victoria,0.0127
3,20000022000,20110,Alpine (S),2,Victoria,0.0105
4,20000023000,20110,Alpine (S),2,Victoria,0.0105


In [50]:
df_lga = df_lga[['LGA_CODE_2019', 'LGA_NAME_2019']]
df_lga.head()

,LGA_CODE_2019,LGA_NAME_2019
0,29499,No usual address (Vic.)
1,20110,Alpine (S)
2,20110,Alpine (S)
3,20110,Alpine (S)
4,20110,Alpine (S)


In [51]:
unique_lga_codes = df_lga['LGA_CODE_2019'].unique()
unique_lga_names = df_lga['LGA_NAME_2019'].unique()
df_lga_unique = pd.DataFrame({'lga_code':unique_lga_codes, 'lga_name':unique_lga_names})

In [52]:
print("Shape: ", df_lga_unique.shape)
df_lga_unique.head()

Shape:  (82, 2)


,lga_code,lga_name
0,29499,No usual address (Vic.)
1,20110,Alpine (S)
2,20260,Ararat (RC)
3,25490,Moyne (S)
4,20570,Ballarat (C)


### Read in GCP data and add the LGA names to it

Read in the Excel file, then remove the 'LGA' prefix from each LGA code in the GCP data

In [53]:
df_gcp = pd.read_excel('gcp_selected.xlsx')

In [54]:
print('Shape: ', df_gcp.shape)
df_gcp.head()

Shape:  (82, 16)


,LGA_CODE_2016,Tot_P_F,Median_tot_fam_inc_weekly,F_650_799_Tot,F_800_999_Tot,F_1000_1249_Tot,F_1250_1499_Tot,F_1500_1749_Tot,F_1750_1999_Tot,F_2000_2999_Tot,F_3000_more_Tot,Percnt_Employment_to_populn_F,F_PGrad_Deg_Total,F_GradDip_and_GradCert_Total,F_BachDeg_Total,F_AdvDip_and_Dip_Total
0,LGA20110,6265,1322,489,404,294,174,139,67,65,39,50.4,123,179,700,603
1,LGA20260,5449,1263,438,361,252,154,118,60,48,42,48.4,57,118,492,470
2,LGA20570,52821,1489,3842,3329,2953,1851,1324,783,702,400,50.8,1418,1492,6296,4114
3,LGA20660,62644,2033,3942,4152,4290,3115,2548,1504,1753,875,54.9,3704,2443,11312,5358
4,LGA20740,16912,1192,1257,934,698,367,295,154,156,98,42.4,299,389,1556,1547


In [55]:
df_gcp['LGA_CODE_2016'].replace('LGA', '', inplace = True, regex = True)

In [56]:
df_gcp.head()

,LGA_CODE_2016,Tot_P_F,Median_tot_fam_inc_weekly,F_650_799_Tot,F_800_999_Tot,F_1000_1249_Tot,F_1250_1499_Tot,F_1500_1749_Tot,F_1750_1999_Tot,F_2000_2999_Tot,F_3000_more_Tot,Percnt_Employment_to_populn_F,F_PGrad_Deg_Total,F_GradDip_and_GradCert_Total,F_BachDeg_Total,F_AdvDip_and_Dip_Total
0,20110,6265,1322,489,404,294,174,139,67,65,39,50.4,123,179,700,603
1,20260,5449,1263,438,361,252,154,118,60,48,42,48.4,57,118,492,470
2,20570,52821,1489,3842,3329,2953,1851,1324,783,702,400,50.8,1418,1492,6296,4114
3,20660,62644,2033,3942,4152,4290,3115,2548,1504,1753,875,54.9,3704,2443,11312,5358
4,20740,16912,1192,1257,934,698,367,295,154,156,98,42.4,299,389,1556,1547


Merge LGA names df with GCP df. Merge by LGA code

In [57]:
# convert LGA_CODE_2016 to int so it can be compared to lga_code, which is an int
df_gcp['LGA_CODE_2016'] = df_gcp['LGA_CODE_2016'].astype("int64")
df_gcp = pd.merge(df_lga_unique, df_gcp, left_on = 'lga_code', right_on = 'LGA_CODE_2016')

In [58]:
print('Shape: ', df_gcp.shape)
df_gcp.head()

Shape:  (82, 18)


,lga_code,lga_name,LGA_CODE_2016,Tot_P_F,Median_tot_fam_inc_weekly,F_650_799_Tot,F_800_999_Tot,F_1000_1249_Tot,F_1250_1499_Tot,F_1500_1749_Tot,F_1750_1999_Tot,F_2000_2999_Tot,F_3000_more_Tot,Percnt_Employment_to_populn_F,F_PGrad_Deg_Total,F_GradDip_and_GradCert_Total,F_BachDeg_Total,F_AdvDip_and_Dip_Total
0,29499,No usual address (Vic.),29499,3238,0,162,136,118,70,41,28,30,23,27.4,106,60,449,219
1,20110,Alpine (S),20110,6265,1322,489,404,294,174,139,67,65,39,50.4,123,179,700,603
2,20260,Ararat (RC),20260,5449,1263,438,361,252,154,118,60,48,42,48.4,57,118,492,470
3,25490,Moyne (S),25490,8245,1486,553,503,451,265,177,101,119,83,53.9,150,189,888,740
4,20570,Ballarat (C),20570,52821,1489,3842,3329,2953,1851,1324,783,702,400,50.8,1418,1492,6296,4114


Remove duplicate column of LGA codes

In [59]:
df_gcp.drop('LGA_CODE_2016', axis = 1, inplace = True)

In [60]:
df_gcp.head()

,lga_code,lga_name,Tot_P_F,Median_tot_fam_inc_weekly,F_650_799_Tot,F_800_999_Tot,F_1000_1249_Tot,F_1250_1499_Tot,F_1500_1749_Tot,F_1750_1999_Tot,F_2000_2999_Tot,F_3000_more_Tot,Percnt_Employment_to_populn_F,F_PGrad_Deg_Total,F_GradDip_and_GradCert_Total,F_BachDeg_Total,F_AdvDip_and_Dip_Total
0,29499,No usual address (Vic.),3238,0,162,136,118,70,41,28,30,23,27.4,106,60,449,219
1,20110,Alpine (S),6265,1322,489,404,294,174,139,67,65,39,50.4,123,179,700,603
2,20260,Ararat (RC),5449,1263,438,361,252,154,118,60,48,42,48.4,57,118,492,470
3,25490,Moyne (S),8245,1486,553,503,451,265,177,101,119,83,53.9,150,189,888,740
4,20570,Ballarat (C),52821,1489,3842,3329,2953,1851,1324,783,702,400,50.8,1418,1492,6296,4114


### Remove non-LGAs
Remove rows where the LGA code starts with '29' - these are not incorporated LGAs and are not included in the shapefile.

Three rows will end up being deleted: No usual address (29499), offshore shipping (29799), unincorporated (29399)

In [61]:
df_uninc = df_gcp['lga_code'][df_gcp['lga_code'] >= 29000]
df_uninc

0     29499
23    29399
81    29799
Name: lga_code, dtype: int64

In [62]:
df_gcp.drop(df_uninc.index.values, inplace = True)
df_gcp.sort_values(by = 'lga_code', inplace = True)
df_gcp.reset_index(drop = True, inplace = True)
print('Shape: ', df_gcp.shape)
df_gcp.head()

Shape:  (79, 17)


,lga_code,lga_name,Tot_P_F,Median_tot_fam_inc_weekly,F_650_799_Tot,F_800_999_Tot,F_1000_1249_Tot,F_1250_1499_Tot,F_1500_1749_Tot,F_1750_1999_Tot,F_2000_2999_Tot,F_3000_more_Tot,Percnt_Employment_to_populn_F,F_PGrad_Deg_Total,F_GradDip_and_GradCert_Total,F_BachDeg_Total,F_AdvDip_and_Dip_Total
0,20110,Alpine (S),6265,1322,489,404,294,174,139,67,65,39,50.4,123,179,700,603
1,20260,Ararat (RC),5449,1263,438,361,252,154,118,60,48,42,48.4,57,118,492,470
2,20570,Ballarat (C),52821,1489,3842,3329,2953,1851,1324,783,702,400,50.8,1418,1492,6296,4114
3,20660,Banyule (C),62644,2033,3942,4152,4290,3115,2548,1504,1753,875,54.9,3704,2443,11312,5358
4,20740,Bass Coast (S),16912,1192,1257,934,698,367,295,154,156,98,42.4,299,389,1556,1547


### Combine income columns into one

Combine all the income counts into one column that represents the percentage of women in the LGA who are earning more than $650 per week.

First, use a regular expression to put the income column names into a list.

In [63]:
pattern = re.compile('F_[0-9]+_[0-9]+_Tot')

In [64]:
column_names = df_gcp.columns.values

In [65]:
income_names = []

for name in column_names:
    if pattern.match(name) != None:
        income_names.append(name)

# add the last income column name, which doesn't match the regex
income_names.append('F_3000_more_Tot')
income_names

['F_650_799_Tot',
 'F_800_999_Tot',
 'F_1000_1249_Tot',
 'F_1250_1499_Tot',
 'F_1500_1749_Tot',
 'F_1750_1999_Tot',
 'F_2000_2999_Tot',
 'F_3000_more_Tot']

In [66]:
df_gcp['pcent_women_high_income'] = df_gcp[income_names].sum(axis = 1)/df_gcp['Tot_P_F']*100
df_gcp.head()

,lga_code,lga_name,Tot_P_F,Median_tot_fam_inc_weekly,F_650_799_Tot,F_800_999_Tot,F_1000_1249_Tot,F_1250_1499_Tot,F_1500_1749_Tot,F_1750_1999_Tot,F_2000_2999_Tot,F_3000_more_Tot,Percnt_Employment_to_populn_F,F_PGrad_Deg_Total,F_GradDip_and_GradCert_Total,F_BachDeg_Total,F_AdvDip_and_Dip_Total,pcent_women_high_income
0,20110,Alpine (S),6265,1322,489,404,294,174,139,67,65,39,50.4,123,179,700,603,26.671987
1,20260,Ararat (RC),5449,1263,438,361,252,154,118,60,48,42,48.4,57,118,492,470,27.032483
2,20570,Ballarat (C),52821,1489,3842,3329,2953,1851,1324,783,702,400,50.8,1418,1492,6296,4114,28.746143
3,20660,Banyule (C),62644,2033,3942,4152,4290,3115,2548,1504,1753,875,54.9,3704,2443,11312,5358,35.404827
4,20740,Bass Coast (S),16912,1192,1257,934,698,367,295,154,156,98,42.4,299,389,1556,1547,23.409413


Drop the now unneeded income columns

In [67]:
df_gcp.drop(income_names, axis = 1, inplace = True)

In [68]:
print("Shape:", df_gcp.shape)
df_gcp.head()

Shape: (79, 10)


,lga_code,lga_name,Tot_P_F,Median_tot_fam_inc_weekly,Percnt_Employment_to_populn_F,F_PGrad_Deg_Total,F_GradDip_and_GradCert_Total,F_BachDeg_Total,F_AdvDip_and_Dip_Total,pcent_women_high_income
0,20110,Alpine (S),6265,1322,50.4,123,179,700,603,26.671987
1,20260,Ararat (RC),5449,1263,48.4,57,118,492,470,27.032483
2,20570,Ballarat (C),52821,1489,50.8,1418,1492,6296,4114,28.746143
3,20660,Banyule (C),62644,2033,54.9,3704,2443,11312,5358,35.404827
4,20740,Bass Coast (S),16912,1192,42.4,299,389,1556,1547,23.409413


### Combine education columns into one

Combine all the education counts into one column that represents the percentage of women in the LGA who are tertiary educated.

First, use a regular expression to put the income column names into a list.

In [69]:
pattern = re.compile('F_\w+_Total')

educ_names = []

for name in column_names:
    if pattern.match(name) != None:
        educ_names.append(name)

educ_names

['F_PGrad_Deg_Total',
 'F_GradDip_and_GradCert_Total',
 'F_BachDeg_Total',
 'F_AdvDip_and_Dip_Total']

In [70]:
df_gcp['pcent_women_tertiary_educ'] = df_gcp[educ_names].sum(axis = 1)/df_gcp['Tot_P_F']*100
df_gcp.head()

,lga_code,lga_name,Tot_P_F,Median_tot_fam_inc_weekly,Percnt_Employment_to_populn_F,F_PGrad_Deg_Total,F_GradDip_and_GradCert_Total,F_BachDeg_Total,F_AdvDip_and_Dip_Total,pcent_women_high_income,pcent_women_tertiary_educ
0,20110,Alpine (S),6265,1322,50.4,123,179,700,603,26.671987,25.618516
1,20260,Ararat (RC),5449,1263,48.4,57,118,492,470,27.032483,20.866214
2,20570,Ballarat (C),52821,1489,50.8,1418,1492,6296,4114,28.746143,25.217243
3,20660,Banyule (C),62644,2033,54.9,3704,2443,11312,5358,35.404827,36.423281
4,20740,Bass Coast (S),16912,1192,42.4,299,389,1556,1547,23.409413,22.416036


Drop the now unneeded education columns

In [71]:
df_gcp.drop(educ_names, axis = 1, inplace = True)

In [72]:
df_gcp.head()

,lga_code,lga_name,Tot_P_F,Median_tot_fam_inc_weekly,Percnt_Employment_to_populn_F,pcent_women_high_income,pcent_women_tertiary_educ
0,20110,Alpine (S),6265,1322,50.4,26.671987,25.618516
1,20260,Ararat (RC),5449,1263,48.4,27.032483,20.866214
2,20570,Ballarat (C),52821,1489,50.8,28.746143,25.217243
3,20660,Banyule (C),62644,2033,54.9,35.404827,36.423281
4,20740,Bass Coast (S),16912,1192,42.4,23.409413,22.416036


### Rename some columns to make them easier to use

Make them all lower case, so they're easier to work with

In [73]:
df_gcp.rename(columns = {'Tot_P_F':'total_women',
                         'Median_tot_fam_inc_weekly':'median_fam_inc_weekly',
                         'Percnt_Employment_to_populn_F':'pcent_women_employed'}, inplace = True)

In [74]:
df_gcp.head()

,lga_code,lga_name,total_women,median_fam_inc_weekly,pcent_women_employed,pcent_women_high_income,pcent_women_tertiary_educ
0,20110,Alpine (S),6265,1322,50.4,26.671987,25.618516
1,20260,Ararat (RC),5449,1263,48.4,27.032483,20.866214
2,20570,Ballarat (C),52821,1489,50.8,28.746143,25.217243
3,20660,Banyule (C),62644,2033,54.9,35.404827,36.423281
4,20740,Bass Coast (S),16912,1192,42.4,23.409413,22.416036


### Extract LGA type from LGA name

Create new df_loc that will store coordinates of each LGA.

Then add LGA type to its own column. This will help Nominatim search for each LGA's coordinates

In [75]:
df_loc = df_gcp[['lga_code', 'lga_name']]
df_loc.head()

,lga_code,lga_name
0,20110,Alpine (S)
1,20260,Ararat (RC)
2,20570,Ballarat (C)
3,20660,Banyule (C)
4,20740,Bass Coast (S)


In [76]:
types = {'(C)':'City', '(S)':'Shire', '(RC)':'Rural City', '(B)':'Borough'}
pattern = re.compile('\([A-Z]+\)')

def get_lga_type(lga_name):
    match = pattern.search(lga_name).group()
    return types[match]

In [77]:
df_loc['lga_type'] = df_loc['lga_name'].apply(get_lga_type)

In [78]:
df_loc.head()

,lga_code,lga_name,lga_type
0,20110,Alpine (S),Shire
1,20260,Ararat (RC),Rural City
2,20570,Ballarat (C),City
3,20660,Banyule (C),City
4,20740,Bass Coast (S),Shire


Now remove the bracketed abbreviation from the LGA name

In [79]:
df_loc['lga_name'].replace(pattern, '', inplace = True)
df_loc['lga_name'] = df_loc['lga_name'].apply(lambda x: x.strip())
df_loc.sort_values(by = 'lga_code', inplace = True)

df_loc.head()

,lga_code,lga_name,lga_type
0,20110,Alpine,Shire
1,20260,Ararat,Rural City
2,20570,Ballarat,City
3,20660,Banyule,City
4,20740,Bass Coast,Shire


### Get coordinates of each LGA

In [80]:
# @hidden_cell

USER_AGENT = "a_gill_capstone"

In [81]:
geolocator = Nominatim(user_agent = USER_AGENT)

for i in range(len(df_loc)):
    search_term = '{} of {}, Vic, Australia'.format(df_loc.loc[i, 'lga_type'], df_loc.loc[i, 'lga_name'])
    location = geolocator.geocode(search_term)
    
    if (location == None):
        search_term = '{}, Vic, Australia'.format(df_loc.loc[i, 'lga_name'])
        location = geolocator.geocode(search_term)
    
    df_loc.loc[i, 'lga_lat'] = location.latitude
    df_loc.loc[i, 'lga_long'] = location.longitude

In [82]:
df_loc.head()

,lga_code,lga_name,lga_type,lga_lat,lga_long
0,20110,Alpine,Shire,-36.833810,146.987331
1,20260,Ararat,Rural City,-37.489990,142.795854
2,20570,Ballarat,City,-37.519419,143.767585
3,20660,Banyule,City,-37.733876,145.086813
4,20740,Bass Coast,Shire,-38.485470,145.553902


### Get land area of each LGA

This will help set the radius for the Foursquare search

In [83]:
df_area = pd.read_csv("Vic land areas by LGA.csv")

In [84]:
print("Shape:", df_area.shape)
df_area.head()

Shape: (79, 3)


,LGA code,Local Government Area,area_km2
0,20110,Alpine (S),4788.2
1,20260,Ararat (RC),4211.1
2,20570,Ballarat (C),739.0
3,20660,Banyule (C),62.5
4,20740,Bass Coast (S),865.8


Calculate radius of each LGA and add to df_area. Assumes each LGA is a circle, which is okay because Foursquare searches in a circle too.

In [85]:
for i in range(len(df_area)):
    df_area['radius_m'] = (np.sqrt(df_area['area_km2']/np.pi)*1000).round()

# order by LGA code
df_area.sort_values('LGA code', inplace = True)
df_area.head()

,LGA code,Local Government Area,area_km2,radius_m
0,20110,Alpine (S),4788.2,39040.0
1,20260,Ararat (RC),4211.1,36612.0
2,20570,Ballarat (C),739.0,15337.0
3,20660,Banyule (C),62.5,4460.0
4,20740,Bass Coast (S),865.8,16601.0


### Count yoga/pilates studios in each LGA

Details about each studio are not needed; just the number

In [86]:
# @hidden_cell

CLIENT_ID = 'ELZQQXU3E32TKM4PCWLQ1HME2NYWXO5FHN3DNIDWWINMCKFT'
CLIENT_SECRET = 'APVKF1ENQ0EDMKTOEZ43GTBSH1RZMO54UDXTB20RSM1IUGZZ'
VERSION = '20200609'

Call Foursquare API to search each LGA for studios

In [87]:
YOGA_CAT = '4bf58dd8d48988d102941735'
PILATES_CAT = '5744ccdfe4b0c0459246b4b2'
MIN_RADIUS = 7500 # because some LGAs are very small

for i, lat, long, rad in zip(range(len(df_gcp)), df_loc['lga_lat'], df_loc['lga_long'], df_area['radius_m']):
    
    if (rad < MIN_RADIUS):
        rad = MIN_RADIUS;
    
    url = "https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={},{}".format(
    CLIENT_ID, CLIENT_SECRET, VERSION, lat, long, int(rad), YOGA_CAT, PILATES_CAT)
        
    json_response = requests.get(url).json()
            
    df_gcp.loc[i, 'num_studios'] = len(json_response['response']['venues'])

In [88]:
df_gcp.head()

,lga_code,lga_name,total_women,median_fam_inc_weekly,pcent_women_employed,pcent_women_high_income,pcent_women_tertiary_educ,num_studios
0,20110,Alpine (S),6265,1322,50.4,26.671987,25.618516,0.0
1,20260,Ararat (RC),5449,1263,48.4,27.032483,20.866214,0.0
2,20570,Ballarat (C),52821,1489,50.8,28.746143,25.217243,3.0
3,20660,Banyule (C),62644,2033,54.9,35.404827,36.423281,11.0
4,20740,Bass Coast (S),16912,1192,42.4,23.409413,22.416036,0.0


Express num_studios per 10000 population of women to account for LGA population differences

In [89]:
df_gcp['studios_per_10000_women'] = df_gcp['num_studios']/(df_gcp['total_women']/10000)

df_gcp.drop('num_studios', axis = 1, inplace = True)

df_gcp.head()

,lga_code,lga_name,total_women,median_fam_inc_weekly,pcent_women_employed,pcent_women_high_income,pcent_women_tertiary_educ,studios_per_10000_women
0,20110,Alpine (S),6265,1322,50.4,26.671987,25.618516,0.000000
1,20260,Ararat (RC),5449,1263,48.4,27.032483,20.866214,0.000000
2,20570,Ballarat (C),52821,1489,50.8,28.746143,25.217243,0.567956
3,20660,Banyule (C),62644,2033,54.9,35.404827,36.423281,1.755954
4,20740,Bass Coast (S),16912,1192,42.4,23.409413,22.416036,0.000000


### Output dfs so they can be used in a new notebook

In [90]:
df_gcp.to_csv('df_gcp.csv')
df_loc.to_csv('df_loc.csv')
df_area.to_csv('df_area.csv')